#  Auto attendance system using face recognition

#### data set creator

In [4]:
# importing libraries
import cv2
import numpy as np
import sqlite3

In [4]:
# creating table in the database
def createtable(database):
    conn=sqlite3.connect(database)
    conn.execute('''create table students_details
                    (Id int primary key not null,
                    Name text not null,
                    rollno text not null,
                    branch text not null,
                    section text not null)''')

createtable("Students.db")
print("table created succesfully")


table created succesfully


In [ ]:
# creating attendance table in the database
def createtable(database):
    conn=sqlite3.connect(database)
    conn.execute('''create table attendancedetails
                    (Id int primary key not null,
                    period1 text,
                    period2 text,
                    period3 text,
                    period4 text,
                    period5 text,
                    period6 text,
                    period7 text)''')

createtable("attendancedatabase.db")
print("Attendance table is  created succesfully")


In [ ]:
# creating attendance table in the database
def createtable(database):
    conn=sqlite3.connect(database)
    conn.execute('''create table datatime
                    (Id int primary key not null,
                    period1 text,
                    period2 text,
                    period3 text,
                    period4 text,
                    period5 text,
                    period6 text,
                    period7 text)''')

createtable("attendancedatabase.db")
print("datetime table is  created succesfully")


In [2]:
# inserting or updating student details
def insertOrUpdate(ID,name,roll,bran,sec):
    conn = sqlite3.connect("Students.db")
    cmd='select * from students_details where Id='+str(ID)
    c = conn.execute(cmd)
    checkRowExist = -1
    for i in c:
        checkRowExist = 0
    if checkRowExist == 0:
        conn.execute("update students_details set Name =? where Id =?",(name,ID))
        conn.execute("update students_details set rollno =? where Id =?",(roll,ID))
        conn.execute("update students_details set branch =? where Id =?",(bran,ID))
        conn.execute("update students_details set section=? where Id =?",(sec,ID))
        print("updated successfully")
    else:
        conn.execute("insert into students_details(Id,Name,rollno,branch,section) values(?,?,?,?,?)",(ID,name,roll,bran,sec))
        print("inserted successfully")
    conn.commit()
    conn.close()
        
ID = input("Enter your ID:\n")
name = input("Enter your Name:\n")
roll = input("Enter your Rollno:\n")
bran = input("Enter your branch:\n")
sec = input("Enter your section:\n")
insertOrUpdate(ID,name,roll,bran,sec)
video = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("opencv-master\data\haarcascades\haarcascade_frontalface_default.xml")
count = 1
while 1:
    ret,frame = video.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces= face_cascade.detectMultiScale(gray,1.2,5)
    for (x,y,w,h) in faces:
        count = count+1
        cv2.imwrite('Datasets/User.'+str(ID)+'.'+str(count)+'.jpg',gray)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.waitKey(100)
    cv2.imshow('Face',frame)
    cv2.waitKey(1)
    if count>30:
        break
video.release()
cv2.destroyAllWindows()
        


Enter your ID:
1
Enter your Name:
M.Vikash
Enter your Rollno:
174g1a04b3
Enter your branch:
ECE
Enter your section:
B
updated successfully


### Trainer

In [3]:
import os
import numpy as np
import cv2

from PIL import Image

# Some how the method cv2.createLBPHFaceRecognizer() is not working.

recognizer = cv2.face.LBPHFaceRecognizer_create();
path='Datasets'

def getImagewithId(path):
    imagePaths=[os.path.join(path,f) for f in os.listdir(path)]
    faces=[]
    Ids=[]
    for imagePath in imagePaths:
        faceImg=Image.open(imagePath).convert('L')
        #print(faceImg)
        faceNp=np.array(faceImg,'uint8')
        #print(faceNp)
        Id=int(os.path.split(imagePath)[-1].split('.')[1])
        #print(imagePath)
        #print(os.path.split(imagePath))
        faces.append(faceNp)
        #print(Id)
        Ids.append(Id)
        cv2.imshow('Training',faceNp)
        cv2.waitKey(1000)
    return np.array(Ids),faces
getImagewithId(path)
cv2.destroyAllWindows()
Ids,faces=getImagewithId(path)

recognizer.train(faces,Ids)
recognizer.save('recognizer/trainer.yml')
cv2.destroyAllWindows()

[[122 118 124 ... 151 150 161]
 [120 121 128 ... 152 151 158]
 [121 122 126 ... 149 142 141]
 ...
 [ 99  99 104 ... 141 142 143]
 [100 100 105 ... 143 145 146]
 [ 93  97 106 ... 141 142 142]] 1
[[122 125 129 ... 159 156 154]
 [124 129 132 ... 160 159 155]
 [127 132 131 ... 167 164 155]
 ...
 [108 107 103 ... 134 131 131]
 [101 103 105 ... 136 133 136]
 [ 98 100 105 ... 134 130 133]] 1
[[125 125 121 ... 147 153 152]
 [121 123 123 ... 149 153 152]
 [122 123 122 ... 150 151 149]
 ...
 [102 102 103 ... 138 139 137]
 [105 102 102 ... 137 133 128]
 [108 104 104 ... 130 133 138]] 1
[[123 126 121 ... 151 161 160]
 [120 122 121 ... 153 160 158]
 [124 124 125 ... 152 161 162]
 ...
 [104 102  96 ... 137 138 138]
 [108 107 102 ... 131 133 131]
 [106 108 104 ... 134 132 128]] 1
[[123 115 108 ... 148 146 145]
 [116 114 112 ... 146 147 147]
 [117 117 115 ... 146 149 149]
 ...
 [109 106 103 ... 133 137 137]
 [105 102 104 ... 130 137 139]
 [105 102 106 ... 131 137 137]] 1
[[119 121 119 ... 147 140 142]

# Main Program

In [2]:
import cv2,os
import numpy as np
from PIL import Image
import pickle
import sqlite3
from datetime import date,datetime

In [ ]:
def getDetails(Id):
    conn = sqlite3.connect("Students.db")
    cursor = conn.execute("select * from students_details where Id="+str(Id))
    details = None
    for i in cursor:
        details = i
    conn.close()
    return details
def postatt(Id,prd):
    conn=sqlite3.connect('attendancedatabase.db')
    conn.execute('UPDATE attendancedetails SET '+str(prd)+'=?'+'WHERE Id =?',('Present',Id))
    print("attendance posted sucessfully")
    conn.commit()
    conn.close()
def post_date_time(Id,prd):
    conn=sqlite3.connect('attendancedatabase.db')     #detect_types=sqlite3.PARSE_DECLTYPES
    key=0            
    cursor=conn.execute('select '+str(prd)+' from datatime '+'WHERE Id='+str(Id))
    profile =None
    for row in cursor:
        profile=row
    if profile!=None:
        key=1
    if(key == 1):
        conn.execute('UPDATE datatime SET '+str(prd)+'=?'+' WHERE Id=?',(datetime.now(),str(Id)))
    print("time updated successfully")
    conn.commit()
    conn.close()

face_cascade = cv2.CascadeClassifier("opencv-master\data\haarcascades\haarcascade_frontalface_default.xml")
video = cv2.VideoCapture(0)
path = 'Datasets'
recognizer = cv2.face.LBPHFaceRecognizer_create();
recognizer.read("recognizer/trainer.yml")
Id =0
prd=input("\nEnter the period with number to post attendence .\nEx : period1\n")
font=cv2.FONT_HERSHEY_SIMPLEX
while True:
    ret,img = video.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    equ = cv2.equalizeHist(gray) 
    final = cv2.medianBlur(equ, 3)
    faces=face_cascade.detectMultiScale(final,scaleFactor=1.2,minNeighbors=5,minSize=(100,100),flags=cv2.CASCADE_SCALE_IMAGE)
    for(x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        Id,conf=recognizer.predict(gray[y:y+h,x:x+w])
        if conf < 50:
            details=getDetails(str(Id))
            postatt(Id,prd)
            post_date_time(Id,prd)
            if details != None:
                cv2.putText(img,'Hiii '+str(details[1]),(x,y+h+30),cv2.FONT_HERSHEY_TRIPLEX,1,(255,144,30),2)
                cv2.putText(img,'Your Attendence is Posted ',(x,y+h+60),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(30,144,255),2)
                #cv2.putText(img,'At Date & Time :',d_t[0],(x,y+h+60),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,69255),2)
        else:
            cv2.putText(img,"unknown person",(x,y+h+30),cv2.FONT_HERSHEY_TRIPLEX,1,(255,144,30),2)
           
    cv2.imshow('Face',img)
    if cv2.waitKey(20)==ord('q') :
            break
video.release()
cv2.destroyAllWindows()
    


Enter the period with number to post attendence .
Ex : period1
period1
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
ti

attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated succe

attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated succe

attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated succe

attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated successfully
attendance posted sucessfully
time updated succe

In [1]:
import sqlite3
conn = sqlite3.connect("attendancedatabase.db")
c = conn.execute("select * from attendancedetails")
for i in c:
    print(i)